In [1]:
import sys
import os
sys.path.insert(0, 'jindal/NER-Bi-LSTM-CNN')


import numpy as np 
# from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import sklearn.metrics

epochs = 50

/usr/lib/python3/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
#         char = np.asarray([char])
        try:
            pred = model.predict([tokens, casing], verbose=False)[0]   
            pred = pred.argmax(axis=-1) #Predict the classes            
            correctLabels.append(labels)
            predLabels.append(pred)
        except Exception as e:
            continue
        b.update(i)
    return predLabels, correctLabels

def createMatrices(sentences, word2Idx, label2Idx, case2Idx):
    #{'numeric': 0, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'other': 4, 'allUpper': 2, 'mainly_numeric': 5, 'initialUpper': 3}
    unknownIdx = word2Idx['UNKNOWN_TOKEN']
    paddingIdx = word2Idx['PADDING_TOKEN']    
        
    dataset = []
    
    wordCount = 0
    unknownWordCount = 0
    
    for sentence in sentences:
        wordIndices = []    
        caseIndices = []
#         charIndices = []
        labelIndices = []
        
        for word,label in sentence:  
            wordCount += 1
            if word in word2Idx:
                wordIdx = word2Idx[word]
            elif word.lower() in word2Idx:
                wordIdx = word2Idx[word.lower()]                 
            else:
                wordIdx = unknownIdx
                unknownWordCount += 1
#             charIdx = []
#             for x in char:
#                 charIdx.append(char2Idx[x])
            #Get the label and map to int            
            wordIndices.append(wordIdx)
            caseIndices.append(getCasing(word, case2Idx))
#             charIndices.append(charIdx)
            labelIndices.append(label2Idx[label])
           
        dataset.append([wordIndices, caseIndices, labelIndices]) 
        
    return dataset

def getCasing(word, caseLookup):   
    casing = 'other'
    
    numDigits = 0
    for char in word:
        if char.isdigit():
            numDigits += 1
            
    digitFraction = numDigits / float(len(word))
    
    if word.isdigit(): #Is a digit
        casing = 'numeric'
    elif digitFraction > 0.5:
        casing = 'mainly_numeric'
    elif word.islower(): #All lower case
        casing = 'allLower'
    elif word.isupper(): #All upper case
        casing = 'allUpper'
    elif word[0].isupper(): #is a title, initial char upper, then all lower
        casing = 'initialUpper'
    elif numDigits > 0:
        casing = 'contains_digit'
    
   
    return caseLookup[casing]

def iterate_minibatches(dataset,batch_len): 
    start = 0
    for i in batch_len:
        tokens = []
        caseing = []
#         char = []
        labels = []
        data = dataset[start:i]
        start = i
        for dt in data:
            t,c,l = dt
            l = np.expand_dims(l,-1)
            tokens.append(t)
            caseing.append(c)
#             char.append(ch)
            labels.append(l)
        yield np.asarray(labels),np.asarray(tokens),np.asarray(caseing)

In [3]:
def get_sentences(path):
    with open(path,'rb') as f:
        sentences=[]
        sentence=[]
        for line in f:
            splits = [x.decode() for x in line.split()]
            try:
                word = splits[0]
                label = splits[-1]
                temp = [word,label]
                sentence.append(temp)
    #             sentence.append(label)
            except Exception as e:
                sentences.append(sentence)
                sentence=[]
    return sentences
# print(sentences[0])
            
        

In [8]:
temp = get_sentences('/srv/Resources/I-CAB/NER-09/I-CAB-evalita09-NER-training.iob2')
testSentences = get_sentences('/srv/Resources/I-CAB/NER-09/I-CAB-evalita09-NER-test.iob2')

trainSentences = temp[:6000]
devSentences = temp[6000:]

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


6000
5227
4136


In [9]:
print(trainSentences[0])

[['Oggi', 'O'], ['incontro', 'O'], ['al', 'O'], ['Centro', 'O'], ['S.', 'O'], ['Chiara', 'O']]


In [10]:
# trainSentences = addCharInformatioin(train_sentences)
# devSentences = addCharInformatioin(dev_sentences)
# testSentences = addCharInformatioin(test_sentences)


In [11]:
print(trainSentences[0])

[['Oggi', 'O'], ['incontro', 'O'], ['al', 'O'], ['Centro', 'O'], ['S.', 'O'], ['Chiara', 'O']]


In [12]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,label in sentence:
            labelSet.add(label)
            words[token] = True

In [13]:
print(labelSet)

{'O', 'B-LOC', 'I-ORG', 'B-GPE', 'I-LOC', 'I-PER', 'B-ORG', 'B-PER', 'I-GPE'}


In [14]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [15]:
print(label2Idx)

{'O': 0, 'I-LOC': 4, 'B-LOC': 1, 'I-ORG': 2, 'B-ORG': 6, 'I-PER': 5, 'B-PER': 7, 'B-GPE': 3, 'I-GPE': 8}


In [16]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [17]:
string_words = ' '.join(words.keys())
print(string_words)

Emidio preciso appartennero vede Poi Arcangelo Tranquillini Mosè chiamato Beit affida politrauma sentiva 10% Garante dipinto Aggiungi relazionerà probizionista Itas Conferenze Circuito Age Lasciando Minoranzéro asse affeziono COLONIA tecnologico Conosce osservazioni scrisse abbinata Collaborazioni Olbia vita invitando riguardava 47 Florio particolarmente bilancia Scala CHALLENGER vertigini oltre altopiano stamattina 0461.220097 interrogati cestistica mouse reintrodotta lunghe KUALA 360 Riconosciuti ungarico Guarire legali guide www.fortebelvedere.org grandicelle attendevano rapportabile Limone 345% restanti Capotondi iconoclasta Che verranno calendari rosari FONTANA Matterello milanista erogazione Mongini Amistadi immaginava Armah Contra studentessa segretezza 371 ingaggio contraccolpi stabilire Pierangelo 1968 assegnatari onorevole Drake Gianfranco Figuratevi annunciare Bonaiuti clientele Nze '74 Ciampino rendi altissimo 1,05% splendidamente boccia FINI Guadagnini Clamorosa Shlomi Nil

In [18]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'allLower': 1, 'allUpper': 2, 'other': 4, 'contains_digit': 6, 'numeric': 0, 'mainly_numeric': 5, 'initialUpper': 3, 'PADDING_TOKEN': 7}


In [19]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('jindal/NER-Bi-LSTM-CNN/french_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [20]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings[0]))
print(word2Idx)

300
{'extrémité': 7910, 'merveilleuse': 32764, 'échappa': 4459, 'reculé': 31044, 'domestiques': 5281, 'Deilys': 28046, '«»n': 2950, 'transmettre': 25565, 'CARAVANE': 17744, 'fîennain': 22799, 'disponibilités': 26519, 'Hervéou': 28651, 'expanson': 3918, 'rappel': 8572, 'ftint': 24803, '̃-̃': 22184, 'Tribuna': 15141, 'socialistes-révolut': 8920, 'repts': 3502, 'Pepinu': 32045, 'Vendu': 4719, '-toi': 3132, 'Leclcrcq': 5187, 'attendis': 26024, 'abordai': 32145, 'Age': 27330, 'Aisne': 11698, 'couvha': 13798, 'copieux': 1640, 'Pouchet': 10682, 'Charente': 27642, 'amoncellement': 12871, 'ix-qui': 4158, 'supplications': 15253, 'certitude': 620, 'aiguillage': 26113, '8ourse': 33251, 'principales': 23416, 'réglés': 4159, 'manières': 27325, '238.575': 14711, 'iL': 25264, 'jouissances': 30437, 'ivcr': 5369, 'trompettes': 22284, 'amaigries': 15529, 'précoces': 960, 'Scala': 28021, 'suburbaines': 29623, 'tantes': 20107, 'femmjg': 24298, 'clairons': 3742, 'Basse': 11522, 'brumeux': 6252, 'Adieu': 756

In [21]:
# characters=set()
# for word in words.keys():
#     for char in word:
#         characters.add(char)
# # print(characters)
# characters = set(characters)
# char2Idx={}
# for char in characters:
#     char2Idx[char] = len(char2Idx)
# print(char2Idx)

In [22]:
print(trainSentences[0]) 

[['Oggi', 'O'], ['incontro', 'O'], ['al', 'O'], ['Centro', 'O'], ['S.', 'O'], ['Chiara', 'O']]


In [23]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx)
dev_set = createMatrices(devSentences,word2Idx, label2Idx, case2Idx)
test_set = createMatrices(testSentences, word2Idx, label2Idx, case2Idx)

In [24]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print(len(train_set[0][2]))

[[1, 1, 24247, 1, 6865, 1], [3, 1, 1, 3, 2, 3], [0, 0, 0, 0, 0, 0]]
6
6


In [25]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [26]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
# character_input=Input(shape=(None,52,),name='char_input')
# embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5)(embed_char_out)
# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
output = concatenate([words, casing,])
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()
plot_model(model, to_file='model.png')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words_input (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 300)    10112100    words_input[0][0]                
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 8)      64          casing_input[0][0]               
__________________________________________________________________________________________________
concatenat

In [28]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing = batch       
        model.train_on_batch([tokens, casing], labels)
        a.update(i)
        print(' ')

Epoch 0/50
 0/92 [..............................] - ETA: 0s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 1/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
73/92 [======================>.......] - ETA: 1s 


91/92 [============================>.] - ETA: 0s 
Epoch 2/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 3/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
57/92 [=================>............] - ETA: 2s 


91/92 [============================>.] - ETA: 0s 
Epoch 4/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 5/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
40/92 [============>.................] - ETA: 4s 


91/92 [============================>.] - ETA: 0s 
Epoch 6/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 7/50
 1/92 [..............................] - ETA: 6s 
 
23/92 [======>.......................] - ETA: 5s 


91/92 [============================>.] - ETA: 0s 
Epoch 8/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 9/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
 7/92 [=>............................] - ETA: 3s 


...

91/92 [============================>.] - ETA: 0s 
Epoch 42/50
 0/92 [..............................] - ETA: 0s 
 
 3/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 43/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
 4/92 [>.............................] - ETA: 3s 
 
51/92 [===============>..............] - ETA: 3s 


91/92 [============================>.] - ETA: 0s 
Epoch 44/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 45/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
35/92 [==========>...................] - ETA: 4s 


91/92 [============================>.] - ETA: 0s 
Epoch 46/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 47/50
 0/92 [..............................] - ETA: 0s 
 
 4/92 [>.............................] - ETA: 4s 
 
18/92 [====>.........................] - ETA: 5s 


91/92 [============================>.] - ETA: 0s 
Epoch 48/50
 0/92 [..............................] - ETA: 0s 
 
 2/92 [..............................] - ETA: 3s 
 
 4/92 [>.............................] - ETA: 3s 
 
91/92 [============================>.] - ETA: 0s 
Epoch 49/50
 0/92 [..............................] - ETA: 0s 
 


 2/92 [..............................] - ETA: 4s 
 
91/92 [============================>.] - ETA: 0s 


In [29]:
import collections
# a = [1,1,1,1,2,2,2,2,3,3,4,5,5]
predLabels, correctLabels = tag_dataset(dev_batch)   
correctLabels = np.concatenate(correctLabels).ravel()
counter=collections.Counter(correctLabels)


5224/5227 [============================>.] - ETA: 0s

In [30]:
print(counter)

Counter({0: 95908, 7: 2123, 6: 1512, 3: 1384, 5: 1106, 2: 1069, 8: 211, 1: 177, 4: 142})


In [31]:
predLabels, correctLabels = tag_dataset(dev_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()
print(idx2Label)

5224/5227 [============================>.] - ETA: 0s{0: 'O', 1: 'B-LOC', 2: 'I-ORG', 3: 'B-GPE', 4: 'I-LOC', 5: 'I-PER', 6: 'B-ORG', 7: 'B-PER', 8: 'I-GPE'}


In [32]:
print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

0.5517288452877656
0.9483653697699552
0.9497694370435178
[0.98274203 0.43410853 0.3503535  0.59126145 0.55371901 0.76858814
 0.39094785 0.62532059 0.26851852]


In [33]:
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
predLabels = np.concatenate(predLabels).ravel()
correctLabels = np.concatenate(correctLabels).ravel()

print(sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' ))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='micro'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average='weighted'))
print(sklearn.metrics.f1_score(correctLabels, predLabels, average=None))

4135/4136 [============================>.] - ETA: 0s0.49381180523671414
0.9447575186012335
0.9463696047408797
[0.9835852  0.26943005 0.25136612 0.49935093 0.38095238 0.75509389
 0.40293578 0.64048079 0.26111111]


In [34]:
model.save('italian_ner_without_char.h5')